<a href="https://colab.research.google.com/github/akashb1504/Hybrid-Email-spam-detection/blob/main/NLP_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
meruvulikith_190k_spam_ham_email_dataset_for_classification_path = kagglehub.dataset_download('meruvulikith/190k-spam-ham-email-dataset-for-classification')

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report


nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df=pd.read_csv("/kaggle/input/190k-spam-ham-email-dataset-for-classification/spam_Emails_data.csv")

In [ ]:
df

,label,text
0,Spam,viiiiiiagraaaa\nonly for the ones that want to...
1,Ham,got ice thought look az original message ice o...
2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...
3,Spam,start increasing your odds of success & live s...
4,Ham,author jra date escapenumber escapenumber esca...
...,...,...
193847,Ham,on escapenumber escapenumber escapenumber rob ...
193848,Spam,we have everything you need escapelong cialesc...
193849,Ham,hi quick question say i have a date variable i...
193850,Spam,thank you for your loan request which we recie...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193852 entries, 0 to 193851
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   193852 non-null  object
 1   text    193850 non-null  object
dtypes: object(2)
memory usage: 3.0+ MB


In [ ]:
df = df.dropna(subset=['text'])

In [ ]:
df.shape

(193850, 2)

# Text Cleaning (Lowercase, Remove Punctuations, Remove Stopwords)

In [ ]:
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

df = df.copy()
df['clean_text'] = df['text'].apply(clean_text)

# Label Encoding

In [ ]:
df['label_encoded'] = df['label'].map({'Ham': 0, 'Spam': 1})

In [ ]:
df

,label,text,clean_text,label_encoded
0,Spam,viiiiiiagraaaa\nonly for the ones that want to...,viiiiiiagraaaa ones want make scream prodigy s...,1
1,Ham,got ice thought look az original message ice o...,got ice thought look az original message ice o...,0
2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...,yo ur wom ne eds escapenumber ch n b e th n f ...,1
3,Spam,start increasing your odds of success & live s...,start increasing odds success live sexually he...,1
4,Ham,author jra date escapenumber escapenumber esca...,author jra date escapenumber escapenumber esca...,0
...,...,...,...,...
193847,Ham,on escapenumber escapenumber escapenumber rob ...,escapenumber escapenumber escapenumber rob dix...,0
193848,Spam,we have everything you need escapelong cialesc...,everything need escapelong cialescapenumbers s...,1
193849,Ham,hi quick question say i have a date variable i...,hi quick question say date variable data frame...,0
193850,Spam,thank you for your loan request which we recie...,thank loan request recieved escapenumber escap...,1


In [ ]:
label_counts = df['label_encoded'].value_counts()
print(label_counts)


label_encoded
0    102159
1     91691
Name: count, dtype: int64


# TF-IDF Vectorization (For Shallow Models)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(df['clean_text']).toarray()

# Tokenization and Padding (For LSTM)

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['clean_text'])

X_seq = tokenizer.texts_to_sequences(df['clean_text'])
X_seq_padded = pad_sequences(X_seq, maxlen=100)


# Train / Validation / Test Split

In [ ]:
y = df['label_encoded'].values

# For RF
X_temp_tfidf, X_test_tfidf, y_temp, y_test = train_test_split(X_tfidf, y, test_size=0.3, stratify=y, random_state=42)
X_train_tfidf, X_val_tfidf, y_train, y_val = train_test_split(X_temp_tfidf, y_temp, test_size=0.4286, stratify=y_temp, random_state=42)

# For LSTM
X_temp_seq, X_test_seq, _, _ = train_test_split(X_seq_padded, y, test_size=0.3, stratify=y, random_state=42)
X_train_seq, X_val_seq, _, _ = train_test_split(X_temp_seq, y_temp, test_size=0.4286, stratify=y_temp, random_state=42)


# Train Random Forest on TF-IDF

In [ ]:
# Vectorize using top 3000 frequent words
tfidf_vectorizer = TfidfVectorizer(max_features=3000)
X_tfidf = tfidf_vectorizer.fit_transform(df['clean_text'])


rf_model = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=42)
rf_model.fit(X_train_tfidf, y_train)

val_probs = rf_model.predict_proba(X_val_tfidf) #for confidence verifier
val_preds = rf_model.predict(X_val_tfidf)


In [ ]:
print("Validation Accuracy:", accuracy_score(y_val, val_preds))
print("\nClassification Report:\n", classification_report(y_val, val_preds))

Validation Accuracy: 0.9783180591138086

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98     30650
           1       0.98      0.97      0.98     27509

    accuracy                           0.98     58159
   macro avg       0.98      0.98      0.98     58159
weighted avg       0.98      0.98      0.98     58159



In [ ]:
cm = confusion_matrix(y_val, val_preds)
TN, FP, FN, TP = cm.ravel()


TPR = TP / (TP + FN)
TNR = TN / (TN + FP)


print(f"\n True Positive Rate (TPR): {TPR:.4f}")
print(f" True Negative Rate (TNR): {TNR:.4f}")




 True Positive Rate (TPR): 0.9735
 True Negative Rate (TNR): 0.9826


# Confidence Verifier

In [ ]:
confidence_threshold = 0.8


In [ ]:
val_probs = rf_model.predict_proba(X_val_tfidf)

# Predicted class and its confidence (max probability from the RF)
val_pred_classes = np.argmax(val_probs, axis=1)
val_pred_confidence = np.max(val_probs, axis=1)


In [ ]:
# Create masks
accepted_mask = val_pred_confidence >= confidence_threshold
rejected_mask = ~accepted_mask

# Accepted predictions (confident)
X_val_accepted = X_val_tfidf[accepted_mask]
y_val_accepted = y_val[accepted_mask]
preds_accepted = val_pred_classes[accepted_mask]

# Rejected predictions (for LSTM)
X_val_rejected_tfidf = X_val_tfidf[rejected_mask]
X_val_rejected_seq = X_val_seq[rejected_mask]  # This assumes you've got padded sequences
y_val_rejected = y_val[rejected_mask]



In [ ]:
print(f"Total validation samples: {len(y_val)}")
print(f"Accepted by Shallow Classifier: {len(y_val_accepted)}")
print(f"Rejected (goes to Deep Classifier): {len(y_val_rejected)}")


Total validation samples: 58159
Accepted by Shallow Classifier: 51247
Rejected (goes to Deep Classifier): 6912


In [ ]:
import numpy as np

# Check class distribution in rejected validation samples
unique, counts = np.unique(y_val_rejected, return_counts=True)
rejected_class_distribution = dict(zip(unique, counts))

print(" Rejected Validation Set Class Distribution:")
for label, count in rejected_class_distribution.items():
    print(f"Class {label}: {count}")


📊 Rejected Validation Set Class Distribution:
Class 0: 2873
Class 1: 4039


In [ ]:
from sklearn.utils import resample
import numpy as np

# Separate class indices
class_0_indices = np.where(y_val_rejected == 0)[0]
class_1_indices = np.where(y_val_rejected == 1)[0]

# Undersample class 1 to match class 0
undersampled_class_1_indices = resample(
    class_1_indices,
    replace=False,
    n_samples=len(class_0_indices),
    random_state=42
)

# Combine and shuffle
balanced_indices = np.concatenate([class_0_indices, undersampled_class_1_indices])
np.random.shuffle(balanced_indices)

# Apply to sequences and labels
X_val_rejected_seq_balanced = X_val_rejected_seq[balanced_indices]
y_val_rejected_balanced = y_val_rejected[balanced_indices]


In [ ]:
import numpy as np

# Check class distribution
class_counts = np.bincount(y_val_rejected_balanced)
print(f"Class 0 (Ham): {class_counts[0]}")
print(f"Class 1 (Spam): {class_counts[1]}")


Class 0 (Ham): 2873
Class 1 (Spam): 2873


# LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop

# Create the LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=30000, output_dim=128))
lstm_model.add(LSTM(64))
lstm_model.add(Dense(256, activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
optimizer = RMSprop(learning_rate=0.001, rho=0.9)
lstm_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

lstm_model.fit(
    X_val_rejected_seq_balanced,
    y_val_rejected_balanced,
    epochs=50,
    batch_size=128,
    validation_split=0.1,
    verbose=1
)


Epoch 1/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5187 - loss: 0.6915 - val_accuracy: 0.5704 - val_loss: 0.6761
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6106 - loss: 0.6607 - val_accuracy: 0.7078 - val_loss: 0.5816
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7645 - loss: 0.5165 - val_accuracy: 0.7965 - val_loss: 0.4440
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8660 - loss: 0.3330 - val_accuracy: 0.7843 - val_loss: 0.4682
Epoch 5/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9017 - loss: 0.2530 - val_accuracy: 0.8313 - val_loss: 0.3977
Epoch 6/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9329 - loss: 0.1766 - val_accuracy: 0.8226 - val_loss: 0.4290
Epoch 7/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9473 - loss: 0.1474 - val_accuracy: 0.8452 - val_loss: 0.4732
Epoch 8/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9605 - loss: 0.1031 - val_accuracy: 0.8296 - val_loss

In [ ]:
# LSTM Predict
lstm_probs = lstm_model.predict(X_val_rejected_seq)
lstm_pred_classes = (lstm_probs >= 0.5).astype(int).reshape(-1)

# Confidence threshold (same as paper or RF)
lstm_confidence_threshold = 0.8
lstm_pred_confidence = lstm_probs.reshape(-1)

# Filter only high-confidence predictions from LSTM
lstm_accepted_mask = lstm_pred_confidence >= lstm_confidence_threshold

# Final LSTM predictions and corresponding true labels
lstm_final_preds = lstm_pred_classes[lstm_accepted_mask]
y_lstm_final = y_val_rejected[lstm_accepted_mask]


216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

lstm_probs = lstm_model.predict(X_val_rejected_seq)
lstm_pred_classes = (lstm_probs >= 0.5).astype(int).reshape(-1)

# Metrics
print(" LSTM Accuracy on Rejected Samples:", accuracy_score(y_val_rejected, lstm_pred_classes))
print("\n LSTM Classification Report on Rejected Samples:")
print(classification_report(y_val_rejected, lstm_pred_classes, zero_division=0))

cm = confusion_matrix(y_val_rejected, lstm_pred_classes)
TN, FP, FN, TP = cm.ravel()

TPR = TP / (TP + FN) if (TP + FN) != 0 else 0
TNR = TN / (TN + FP) if (TN + FP) != 0 else 0

print(f"\n True Positive Rate (TPR): {TPR:.4f}")
print(f" True Negative Rate (TNR): {TNR:.4f}")


216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
 LSTM Accuracy on Rejected Samples: 0.95703125

 LSTM Classification Report on Rejected Samples:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      2873
           1       0.99      0.94      0.96      4039

    accuracy                           0.96      6912
   macro avg       0.95      0.96      0.96      6912
weighted avg       0.96      0.96      0.96      6912


 True Positive Rate (TPR): 0.9364
 True Negative Rate (TNR): 0.9861
